<a href="https://colab.research.google.com/github/shashank0304/ai_agent_book_camp/blob/master/Copy_of_Final2_Vizuara_Agents_Course_Code_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision Agents with smolagents


## Let's install the dependencies and login to our HF account to access the Inference API

If you haven't installed `smolagents` yet, you can do so by running the following command:

In [ ]:
!pip install smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.2/138.2 kB 6.2 MB/s eta 0:00:00


Let's also login to the Hugging Face Hub to have access to the Inference API.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Providing Images at the Start of the Agent's Execution

In this approach, images are passed to the agent at the start and stored as `task_images` alongside the task prompt. The agent then processes these images throughout its execution.  

Consider the case where Alfred wants to verify the identities of the superheroes attending the party. He already has a dataset of images from previous parties with the names of the guests. Given a new visitor's image, the agent can compare it with the existing dataset and make a decision about letting them in.  

In this case, a guest is trying to enter, and Alfred suspects that this visitor might be The Joker impersonating Wonder Woman. Alfred needs to verify their identity to prevent anyone unwanted from entering.  

Let’s build the example. First, the images are loaded. In this case, we use images from Wikipedia to keep the example minimal, but imagine the possible use-cases!

In [ ]:
from PIL import Image
import requests
from io import BytesIO

image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/e/e8/The_Joker_at_Wax_Museum_Plus.jpg",
    "https://upload.wikimedia.org/wikipedia/en/9/98/Joker_%28DC_Comics_character%29.jpg"
]

images = []
for url in image_urls:
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }
    response = requests.get(url,headers=headers)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    images.append(image)

Now that we have the images, the agent will tell us wether the guests is actually a superhero (Wonder Woman) or a villian (The Joker).

In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
!pip install smolagents[telemetry] opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

In [ ]:
import base64
import os

os.environ["LANGFUSE_PUBLIC_KEY"] = ""
os.environ["LANGFUSE_SECRET_KEY"] = ""

LANGFUSE_PUBLIC_KEY = os.environ.get("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.environ.get("LANGFUSE_SECRET_KEY")

if not LANGFUSE_PUBLIC_KEY or not LANGFUSE_SECRET_KEY:
    raise ValueError("Langfuse public or secret keys are missing!")

LANGFUSE_AUTH = base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

#os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

print("Langfuse environment variables set successfully!")


Langfuse environment variables set successfully!


In [ ]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [ ]:
from smolagents import CodeAgent, OpenAIServerModel

model = OpenAIServerModel(model_id="gpt-4o")

# Instantiate the agent
agent = CodeAgent(
    tools=[],
    model=model,
    max_steps=20,
    verbosity_level=2
)

response = agent.run(
    """
    Describe the costume and makeup that the comic character in these photos is wearing and return the description.
    Tell me if the guest is The Joker or Wonder Woman.
    """,
    images=images
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Describe the costume and makeup that the comic character in these photos is wearing and return the description. │
│     Tell me if the guest is The Joker or Wonder Woman.                                                          │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/smolagents/models.py:393: FutureWarning: Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/smolagents/models.py:402: FutureWarning: Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.
  warnings.warn(


Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
The costume and makeup in the photos depict The Joker.                                                             
                                                                                                                   
**Description:**                                                                                                   
                                                                                                                   
1. **Makeup:**                                                                                                     
   - White face paint covers the entire face.                                                                      
   - Bright red lipstick forms a large, exaggerated smile.                                                         
   - Dark eye makeup is used to create a dramatic, sinister look.                                                  
                                                                                                                   
2. **Costume:**                                                                                                    
   - The character is wearing a vibrantly colored suit, typically purple.                                          
   - A bright-colored shirt is visible underneath, often with a contrasting tie or bow.                            
   - Signature green hair is styled back.                                                                          
                                                                                                                   
This distinctive appearance is characteristic of The Joker.                                                        

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
            Here is your code snippet:
            The costume and makeup in the photos depict The Joker.

**Description:**

1. **Makeup:**
   - White face paint covers the entire face.
   - Bright red lipstick forms a large, exaggerated smile.
   - Dark eye makeup is used to create a dramatic, sinister look.

2. **Costume:**
   - The character is wearing a vibrantly colored suit, typically purple.
   - A bright-colored shirt is visible underneath, often with a contrasting tie or bow.
   - Signature green hair is styled back.

This distinctive appearance is characteristic of The Joker.
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            <code>
            # Your python code here
            </code>
Make sure to provide correct code blobs.

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I will describe the costume and makeup of the character in the given images and identify if the character 
is The Joker or Wonder Woman.                                                                                      
                                                                                                                   
<code>                                                                                                             
def describe_costume_and_makeup():                                                                                 
    description = """                                                                                              
    The costume and makeup in the photos depict The Joker.                                                         
                                                                                                                   
    Description:                                                                                                   
    1. Makeup:                                                                                                     
       - White face paint covers the entire face.                                                                  
       - Bright red lipstick forms a large, exaggerated smile.                                                     
       - Dark eye makeup is used to create a dramatic, sinister look.                                              
                                                                                                                   
    2. Costume:                                                                                                    
       - The character is wearing a vibrantly colored suit, typically purple.                                      
       - A bright-colored shirt is visible underneath, often with a contrasting tie or bow.                        
       - Signature green hair is styled back.                                                                      
    """                                                                                                            
    character = "The Joker"                                                                                        
                                                                                                                   
    return description, character                                                                                  
                                                                                                                   
description, character = describe_costume_and_makeup()                                                             
final_answer((description, character))                                                                             
</code>                                                                                                            

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def describe_costume_and_makeup():                                                                               
      description = """                                                                                            
      The costume and makeup in the photos depict The Joker.                                                       
                                                                                                                   
      Description:                                                                                                 
      1. Makeup:                                                                                                   
         - White face paint covers the entire face.                                                                
         - Bright red lipstick forms a large, exaggerated smile.                                                   
         - Dark eye makeup is used to create a dramatic, sinister look.                                            
                                                                                                                   
      2. Costume:                                                                                                  
         - The character is wearing a vibrantly colored suit, typically purple.                                    
         - A bright-colored shirt is visible underneath, often with a contrasting tie or bow.                      
         - Signature green hair is styled back.                                                                    
      """                                                                                                          
      character = "The Joker"                                                                                      
                                                                                                                   
      return description, character                                                                                
                                                                                                                   
  description, character = describe_costume_and_makeup()                                                           
  final_answer((description, character))                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ('\n    The costume and makeup in the photos depict The Joker.\n\n    Description:\n    1. 
Makeup:\n       - White face paint covers the entire face.\n       - Bright red lipstick forms a large, exaggerated
smile.\n       - Dark eye makeup is used to create a dramatic, sinister look.\n\n    2. Costume:\n       - The 
character is wearing a vibrantly colored suit, typically purple.\n       - A bright-colored shirt is visible 
underneath, often with a contrasting tie or bow.\n       - Signature green hair is styled back.\n    ', 'The 
Joker')

[Step 2: Duration 8.57 seconds| Input tokens: 7,243 | Output tokens: 302]

In [ ]:
response

('\n    The costume and makeup in the photos depict The Joker.\n\n    Description:\n    1. Makeup:\n       - White face paint covers the entire face.\n       - Bright red lipstick forms a large, exaggerated smile.\n       - Dark eye makeup is used to create a dramatic, sinister look.\n\n    2. Costume:\n       - The character is wearing a vibrantly colored suit, typically purple.\n       - A bright-colored shirt is visible underneath, often with a contrasting tie or bow.\n       - Signature green hair is styled back.\n    ',
 'The Joker')

In this case, the output reveals that the person is impersonating someone else, so we can prevent The Joker from entering the party!

## Providing Images with Dynamic Retrieval

This examples is provided as a `.py` file since it needs to be run locally since it'll browse the web. Go to the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course) for more details.